# Web Üzerinde İşlemler

* HTTP istekleri
* Uzaktan dosya indirme
* vs.

## Wget yerine Axel Kullanma
Wget basit ve eski bir uygulamadır. Axel yeni ve çok fonksiyoneldir.

Axelin avantajları:

* Paralel olarak indirme
* Hız kısıtlamalarını birden fazla isteklerle ekarde etme (IDM / Torrent gibi)
* Aynı anda çok sayıda bağlantı indirebilme



In [0]:
# Axel indirme
!apt-get install axel

# Axel kullanımı
!axel -h

## Selenium Kullanımı
Selenium web üzerinde (siteye elle girmeden) çok fonksiyonel işlemler yapmamızı sağlar

* Butonlara tıklama
* Bekleme ve hareket etme
* JS çalıştırma vs..


### Selenium Kurulumu
Colab için normalden farklı bir kuruluma ihtiyaç vardır.

In [0]:
!pip install selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin



> **System yoluna bağımlılığı ekleme (chromedriver)**



In [0]:
import sys

# Sistem yoluna chrome-driver'i ekleme
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

## Selenium Örneği
Google üzerinde basit işlemleri yapar:

* Google'a girme
* Başlığı ekrana yazma
* Input kısmına 'cheers!' yazar
* Arama butonuna tıklar
* Çıkan sayfanın başlığını ekrana yazar

In [0]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0

# Chrome ayarlarını oluşturma
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Ayarları belirlenen chrome-driver'ın oluşturulması ve siteye girme
driver = webdriver.Chrome('chromedriver', options=chrome_options)

# go to the google home page
driver.get("http://www.google.com")

# the page is ajaxy so the title is originally this:
print (driver.title)

# find the element that's name attribute is q (the google search box)
inputElement = driver.find_element_by_name("q")

script = "document.getElementsByName('q')[0]"
script1 = "return 5"
print(driver.execute_script("return " + script))

# type in the search
inputElement.send_keys("cheese!")

# submit the form (although google automatically searches now without submitting)
inputElement.submit()

try:
    # we have to wait for the page to refresh, the last thing that seems to be updated is the title
    WebDriverWait(driver, 10).until(EC.title_contains("cheese!"))

    # You should see "cheese! - Google Search"
    print (driver.title)

finally:
    driver.quit()

## Selenium ile İndirme
İşlemin yapılabilmesi için:
.
* Seleniumun yüklü olduğundan emin olma
* *chromium-chromedriver* yüklü olmalı
* *chromedriver*'ın bash'e tanımlı olması

gereklidir.




> **Bağımlılıkları dahil etme**



In [0]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



> **From ile indirme (İsteğe Bağlı)**



> Bu adımı kullanmayıp, fonksiyon ile indirme adımına bakabilirsin





In [0]:
#@title Video Downloader
series = "harrow" #@param ["supernatural", "harrow"] {allow-input: false}
season = 1 #@param {type:"slider", min:1, max:21, step:1}
chapter = 1 #@param {type:"slider", min:1, max:21, step:1}


# Chrome ayarlarını oluşturma
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Ayarları belirlenen chrome-driver'ın oluşturulması
driver = webdriver.Chrome('chromedriver', options=chrome_options)

# URL'in oluşturulması ve ona bağlanma
main_url = "https://www.roketfilmizle1.org/bolum/{}-{}-sezon-{}-bolum-turkce-dublaj-izle".format(series, season, chapter)
driver.get(main_url)

# Başlığı tanımlama
title = driver.title

try:
  # plyg id'li öğe gelene kadar bekleme
  WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "plyg")))

  # Sitede js derleme
  script = "return document.getElementById('plyg').getElementsByTagName('iframe')[0].src"

  # URL'i tanımlama
  url = driver.execute_script(script)
  driver.get(url)

  # Kaynak URL'sini test etme
  isRapidLink = url.find("rapid") > -1

  # Rapid için
  if isRapidLink:
    # Video'nun kaynağını bulma
    script = 'return document.getElementsByTagName("video")[0].getElementsByTagName("source")[0].src'
    video_url = driver.execute_script(script)

  # Oload için    
  else:
    # Ekrana tıklama
    driver.find_element_by_id("videooverlay").click()
    # Video'nun kaynağını bulma
    script = 'return  document.getElementsByTagName("video")[0].currentSrc'
    video_url = driver.execute_script(script)

finally:
  driver.quit()
  
print("Baslık: ", title)
print("URL: ", url)
print("Video URL: ", video_url)



### Selenium Fonksiyonu Tanımlama ve İndirme
Fonksiyon ile toplu indirmeler yapılabilir



> **Dizi isimlerini listeleyen fonksiyon**



In [0]:
def get_series(key='null', show=False, id=-1):
  # Dizi isimleri ve değişkenleri
  dic_series = {
      "Supernatural": "supernatural", 
      "Harrow": "harrow"
  }
  
  arr_series = [
      "supernatural",
      "harrow"
  ]
  
  # İsteğe bağlı gösterme
  if show:
    # Görsellik
    print("İD: Film İsmi -> Anahtarı (key)")
    print("---------------------------")
    
    # Verileri ekrana yazdırma
    i = 0
    for series in dic_series:
      print("{}: {} -> {}".format(i, series, dic_series[series]))
      i += 1
  
  if key != 'null':
    return dic_series[key]
  
  elif id > -1 and id < len(arr_series):
    arr_series = []
    
    for series in dic_series:
      arr_series.append(dic_series[series])
      
    return arr_series[id]
  



> **Verilen sezonu ve bölümü indiren fonksiyon**



In [0]:
def get_video(series, season, chapter):
  # Chrome ayarlarını oluşturma
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')

  # Ayarları belirlenen chrome-driver'ın oluşturulması
  driver = webdriver.Chrome('chromedriver', options=chrome_options)
  
  # URL'in oluşturulması ve ona bağlanma
  main_url = "https://www.roketfilmizle1.org/bolum/{}-{}-sezon-{}-bolum-turkce-dublaj-izle".format(series, season, chapter)
  driver.get(main_url)

  # Başlığı tanımlama
  title = driver.title

  try:
    # plyg id'li öğe gelene kadar bekleme
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "plyg")))

    # Sitede js derleme
    script = "return document.getElementById('plyg').getElementsByTagName('iframe')[0].src"
    
    # URL'i tanımlama
    url = driver.execute_script(script)
    driver.get(url)

    # Kaynak URL'sini test etme
    isRapidLink = url.find("rapid") > -1
    
    # Rapid için
    if isRapidLink:
      # Video'nun kaynağını bulma
      script = 'return document.getElementsByTagName("video")[0].getElementsByTagName("source")[0].src'
      video_url = driver.execute_script(script)
    
    # Oload için    
    else:
      # Ekrana tıklama
      driver.find_element_by_id("videooverlay").click()
      # Video'nun kaynağını bulma
      script = 'return  document.getElementsByTagName("video")[0].currentSrc'
      video_url = driver.execute_script(script)
    
  finally:
    driver.quit()
    
  return title, url, video_url



> **Örnek Kullanım**



In [0]:
get_series(show=True)
get_video(get_series(id=1), 1, 1)



> **Colab ile drive'a erişim sağlama**



In [0]:
if not isMount:
  from google.colab import drive
  drive.mount('/content/mdrive')
  isMount = True



> **Belli sezonlara ait bölümleri indirme**



In [0]:
if isMount:
  for i in range (1, 8):
    !mkdir '/content/mdrive/My Drive/Filmler/Supernatural/{i}.Sezon'
    %cd '/content/mdrive/My Drive/Filmler/Supernatural/{i}.Sezon'

    for j in range (1, 23):
      # 3.sezon 16 bölüm
      if j == 16 and i == 3:
        print("Sezonun son bölümü, sezon atlandı!")
        break

      # 1. sezon indi
      if i == 1:
        break

      !echo "{i}.Sezon {j}.Bolum için indirme linkleri oluşturuluyor.."
      _, main_url, video_url = get_video(get_series(id=0), i, j)
      !echo "{main_url} {video_url}"

      !echo "{i}.Sezon {j}.Bolum indiriliyor.."
      # 20 koldan bağlanma ve sessiz indirme
      !axel -S4 -a -q -c -o "Bolum{j}.mp4" -n 20 "{video_url}"

## HTTP isteği ile url indirme
Film sitelerindeki filmleri uzaktan indirmeyi sağlar


In [0]:
import requests # HTTP istekleri
from bs4 import BeautifulSoup # HTTP isteklerini işleme
import base64 # Şifreleme işlemleri

#@title Roketfilmizle Media Finder

# URL tanımlama
url = 'https://www.roketfilmizle1.org/bolum/supernatural-2-sezon-11-bolum-turkce-dublaj-izle' #@param {type:"string"}

# HTTP isteği ile yanıt alma
r = requests.get(url)

# HTTP yanıtını BeautifulSoup ile işleyecek obje tanımlama
soup = BeautifulSoup(r.text, 'html.parser')

# ID'si verilern html'i elemanın metnini alma
script_text = soup.find(id="plyg").script.get_text()
# Kodlanmış metni alma
encoded_text = str.split('"')[1]
# Kodu çözümleme
decoded_text = base64.b64decode(encoded)

# Kodu çözümlenmiş metni işlemeye hazırlama
soup = BeautifulSoup(decoded_text, 'html.parser')

# URL'i çekme
url = soup.iframe['src']

# Yeni URL ile istekte bulunma
r = requests.get(url)

# HTTP yanıtını BeautifulSoup ile işleyecek obje tanımlama
soup = BeautifulSoup(r.text, 'html.parser')

# document.getElementById("videojs").innerHTML.split('"')[1]

# ID ile HTML elemanını bulma
element = soup.find(id='videojs')

# Elemandan video url'ini alma
media_url = element.source['src']

print(media_url)

### Fonksiyon ile İndirme
Fonksiyon yardımıyla seri bir şekilde videoları indirme

In [0]:
# Verilen sezondaki istenen bölümün video url'ini verir
def get_media_from_roketfilmizle1(season, chapter):
  # HTTP isteğinde bulunma
  url = 'https://www.roketfilmizle1.org/bolum/supernatural-{}-sezon-{}-bolum-turkce-dublaj-izle'.format(season, chapter)
  r = requests.get(url)
  
  # HTTP'yi işleyerek kodlanmış metni alma
  soup = BeautifulSoup(r.text, 'html.parser')
  script_text = soup.find(id="plyg").script.get_text()
  
  # Kodlanmış metni çözümleme
  encoded_text = str.split('"')[1]
  decoded_text = base64.b64decode(encoded)
  
  # HTTP'yi işleyerek yeni url ile istekte bulunma
  soup = BeautifulSoup(decoded_text, 'html.parser')
  url = soup.iframe['src']
  r = requests.get(url)
  
  # HTTP üzerinden video linkini çekme
  soup = BeautifulSoup(r.text, 'html.parser')
  element = soup.find(id='videojs')
  media_url = element.source['src']
  
  return media_url

In [0]:
# Verilen sezonu komple indirir
def get_all_season_from_roketfilmizle1(season):
  for i in 

In [0]:
print(get_media_from_roketfilmizle1(1, 1))
print(get_media_from_roketfilmizle1(1, 22))

## Berkeley Dataset Download

In [0]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [0]:
# Chrome ayarlarını oluşturma
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--safebrowsing-disable-download-protection')
chrome_options.add_argument('--safebrowsing-manual-download-blacklist')

preferences = {"download.default_directory": "/content", "safebrowsing.enabled": "false"}
chrome_options.add_experimental_option("prefs", preferences)

# Ayarları belirlenen chrome-driver'ın oluşturulması
driver = webdriver.Chrome('chromedriver', options=chrome_options)

# URL'in oluşturulması ve ona bağlanma
main_url = "https://bdd-data.berkeley.edu/login.html";
driver.get(main_url)

# Başlığı tanımlama
title = driver.title

try:
  # Sitede js derleme
  script_mail = 'document.getElementById("email").value = "yoif4@alexbox.online"'
  script_pass = 'document.getElementById("password").value = "Temp1234"'
  script_login = 'document.getElementById("login-button").click()'
  
  driver.execute_script(script_mail)
  driver.execute_script(script_pass)
  driver.execute_script(script_login)
  
  WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "download")))
  print(driver.current_url)
  script_download = 'downloadChallengeFile(event, "bdd100k/bdd100k_images.zip")'
  driver.execute_script(script_download)

finally:
  driver.quit()


In [0]:
!ls "/content"